# Tune_lgb
* 제출결과 : 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
import lightgbm as lgb

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

In [2]:
path = "./data/" 
train_df = pd.read_csv(path+'train.csv',index_col=0)
test_df = pd.read_csv(path+'test.csv',index_col=0)
sample_submission_df = pd.read_csv(path+'sample_submission.csv',index_col=0)
pd.options.display.max_columns = 30

## 전처리
 - 이상치 확인 및 처리

In [3]:
train_df.describe()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,199991.000000,1.999910e+05,199991.000000,199991.000000,199991.000000,199991.000000,1.999910e+05,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000
mean,360.830152,-6.750146e+00,18.675373,18.401235,18.043495,17.663526,1.084986e+01,19.072693,19.134483,18.183331,18.000882,21.837903,18.454136,18.481525,17.686617,17.699207,20.110991,18.544375,18.181544,17.692395,17.189281
std,225.305890,1.187678e+04,155.423024,127.128078,116.622194,123.735298,4.172116e+03,749.256162,90.049058,122.378972,145.862346,789.472333,154.376277,97.240448,145.730872,142.691880,122.299062,161.728183,133.984475,131.183416,133.685138
min,1.000000,-5.310802e+06,-40022.466071,-27184.795793,-26566.310827,-24878.828280,-1.864766e+06,-215882.917191,-21802.656144,-20208.516262,-26505.602101,-24463.431833,-25958.752324,-23948.588523,-40438.184078,-30070.729379,-26236.578659,-36902.402336,-36439.638493,-38969.416822,-26050.710196
25%,174.000000,1.965259e+01,18.701180,18.048572,17.747663,17.425523,1.994040e+01,18.902851,18.259352,17.903615,17.606148,19.247795,18.113933,17.479794,17.050294,16.804705,19.266214,18.076120,17.423425,16.977671,16.705774
50%,349.000000,2.087136e+01,19.904235,19.454492,19.043895,18.611799,2.104910e+01,20.069038,19.631419,19.188763,18.710967,20.366848,19.586559,19.182789,18.693370,18.174592,20.406840,19.547674,19.143156,18.641756,18.100997
75%,526.000000,2.216043e+01,21.150297,20.515936,20.073528,19.883760,2.233754e+01,21.385830,20.773911,20.331419,20.133179,21.797480,21.004397,20.457491,20.019112,19.807652,21.992898,20.962386,20.408140,19.968846,19.819554
max,1000.000000,1.877392e+04,3538.984910,3048.110913,4835.218639,9823.740407,4.870154e+03,248077.513380,12084.735440,8059.638535,18358.921741,298771.019041,12139.815877,7003.136546,9772.190537,17403.789263,14488.251976,10582.058590,12237.951703,4062.499371,7420.534172


In [4]:
def del_outlier(data, min=0, max=60):
    up_idx_t=()
    dw_idx_t=()
    train_light = data.iloc[:,2:]
    for i in range(len(train_light.columns)):
        col = train_light.columns[i]
        up_idx_t+=tuple(data[data[col]>max].index)
        dw_idx_t+=tuple(data[data[col]<min].index)
    del_idx = set(up_idx_t+dw_idx_t)
    
    return data[~data.index.isin(del_idx)]

In [5]:
train_df = del_outlier(train_df, min=-40, max=40)

In [6]:
train_df.describe()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000,199646.000000
mean,360.906179,20.981482,19.858757,19.285847,18.870981,18.611269,21.170671,20.089208,19.505094,19.087129,18.828211,20.702285,19.597224,19.005508,18.602769,18.390198,20.728887,19.529152,18.939032,18.519976,18.275557
std,225.353069,2.075314,1.858574,1.703859,1.623027,1.679114,1.965779,1.841201,1.704064,1.624881,1.676159,2.377376,2.062031,1.936239,1.880032,2.014094,2.183187,1.955486,1.858617,1.794479,1.861357
min,1.000000,-12.978725,2.741681,-11.043026,-6.406284,1.762241,-10.838092,-8.294703,8.231789,10.646672,8.049850,-39.027146,-39.777921,-35.878812,-34.494076,-26.350259,3.444448,12.498143,10.286917,11.337467,10.977328
25%,174.000000,19.652024,18.700219,18.048259,17.746887,17.424948,19.941100,18.903197,18.259535,17.904322,17.606707,19.248189,18.113991,17.480514,17.051210,16.805092,19.265102,18.075284,17.423002,16.977550,16.705145
50%,349.000000,20.869344,19.902848,19.453620,19.042646,18.610118,21.048108,20.068700,19.632022,19.189659,18.711177,20.365697,19.585031,19.182702,18.693982,18.173468,20.404358,19.545613,19.142608,18.641821,18.100075
75%,526.000000,22.156051,21.146719,20.514248,20.072455,19.882456,22.335637,21.384566,20.773926,20.331987,20.133465,21.792793,21.001815,20.456142,20.018379,19.806313,21.987909,20.959696,20.407039,19.968263,19.818701
max,1000.000000,38.142419,39.411617,37.348717,34.963620,32.130146,39.182814,37.745406,30.961793,32.171369,36.498179,39.979165,39.969025,39.977525,39.237553,39.794471,39.238248,39.129788,33.666494,32.740051,34.930633


In [7]:
train_df.columns

Index(['type', 'fiberID', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
       'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
       'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
       'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
       'modelMag_z'],
      dtype='object')

## DATA Setting

In [8]:
column_number = {}
number_columns = {}
for i, column in enumerate(sample_submission_df.columns):
    column_number[column] = i
    number_columns[i] = column
    
    
def to_number(x, dic):
    return dic[x]

train_df['type_num'] = train_df['type'].apply(lambda x: to_number(x, column_number))


In [9]:
def add_minus_feature_all(data,test = False):
    '''
    combination으로 모든 변수들의 조합에 대해서 컬럼생성 
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 64개로 증가
    '''
    from itertools import combinations
    n = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
        data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for combi in list(combinations(selected,2)):
            name_1st = str.split(combi[0],'_')[1]
            name_2nd = str.split(combi[1],'_')[1]
            data[mag+"_"+name_1st+"-"+name_2nd] = data[combi[0]]-data[combi[1]]
        n=count
    return data

In [10]:
def add_minus_feature(data,test = False):
    '''
    근접변수에 대해서만 컬럼생성 ex) 'fiberMag_u-g', 'fiberMag_g-r', 'fiberMag_r-i', 'fiberMag_i-z'
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 38개로 증가 
    '''
    from itertools import combinations
    n = 0
    m = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
#         data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for i in range(0,len(selected)):        
            if i>0: 
                name_1st = str.split(m,'_')[1]
                name_2nd = str.split(selected[i],'_')[1]
                data[mag+"_"+name_1st+"-"+name_2nd] = data[m]-data[selected[i]]
            m = selected[i]    
        n=count
    return data

In [11]:
train_df = add_minus_feature(train_df)
test_df = add_minus_feature(test_df,True)

In [12]:
def label_encode(data):
    '''
    각 mag에서 가장 높은값의 filter를 labelencoding 
    '''
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    taget_cols = ['psfMag_MAX','fiberMag_MAX','petroMag_MAX','modelMag_MAX']
    for col in taget_cols:
        data[col] = le.fit_transform(data[col])
    return data

In [13]:
train_df = label_encode(train_df)
test_df = label_encode(test_df)

KeyError: 'psfMag_MAX'

In [ ]:
# train_df.drop(columns=['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
#        'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
#        'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
#        'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
#        'modelMag_z'], inplace=True)

# test_df.drop(columns=['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
#        'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
#        'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
#        'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
#        'modelMag_z'], inplace=True)

In [14]:
train_X = train_df.drop(columns=['type', 'type_num'], axis=1)
train_y = train_df['type_num']

In [15]:
train_X.columns

Index(['fiberID', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z',
       'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z',
       'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i', 'petroMag_z',
       'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z',
       'psfMag_u-g', 'psfMag_g-r', 'psfMag_r-i', 'psfMag_i-z', 'fiberMag_u-g',
       'fiberMag_g-r', 'fiberMag_r-i', 'fiberMag_i-z', 'petroMag_u-g',
       'petroMag_g-r', 'petroMag_r-i', 'petroMag_i-z', 'modelMag_u-g',
       'modelMag_g-r', 'modelMag_r-i', 'modelMag_i-z'],
      dtype='object')

In [17]:
# ## u-g, g-r, r-i, i-z 만 남기는게 좋지 않을까????
# train_X.drop(['psfMag_u-r','psfMag_u-i','psfMag_u-z','psfMag_g-i','psfMag_g-z','psfMag_r-z',
#              'fiberMag_u-r','fiberMag_u-i','fiberMag_u-z','fiberMag_g-i','fiberMag_g-z','fiberMag_r-z',
#              'petroMag_u-r','petroMag_u-i','petroMag_u-z','petroMag_g-i','petroMag_g-z','petroMag_r-z',
#              'modelMag_u-r','modelMag_u-i','modelMag_u-z','modelMag_g-i','modelMag_g-z','modelMag_r-z'] ,axis=1, inplace=True)

# test_X.drop(['psfMag_u-r','psfMag_u-i','psfMag_u-z','psfMag_g-i','psfMag_g-z','psfMag_r-z',
#              'fiberMag_u-r','fiberMag_u-i','fiberMag_u-z','fiberMag_g-i','fiberMag_g-z','fiberMag_r-z',
#              'petroMag_u-r','petroMag_u-i','petroMag_u-z','petroMag_g-i','petroMag_g-z','petroMag_r-z',
#              'modelMag_u-r','modelMag_u-i','modelMag_u-z','modelMag_g-i','modelMag_g-z','modelMag_r-z'] ,axis=1, inplace=True)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.3, random_state=1059,stratify = train_y)

## Modeling

In [17]:
# fit a lightGBM model to the data
model = lgb.LGBMClassifier(
    learning_rate= 0.0015,
    max_depth= 8,
    num_leaves= 120,
    metric= ['multi_logloss', 'multi_logloss'],
    n_estimators= 10000,
    objective= 'multiclass',
    num_class= [train_y.nunique()],
    random_state= 1059,
    reg_alpha= 0.0,
    reg_lambda= 0.0,
    subsample_for_bin= 50000,
    bagging_fraction= 0.85,
    bagging_freq= 20,
    feature_fraction= 0.85,
    )

In [19]:
model.fit(X_train, y_train,
          sample_weight=None, init_score=None, eval_set=[(X_train, y_train),(X_test, y_test)], eval_names=['Train', 'Test'],
          eval_sample_weight=None, eval_class_weight=None, eval_init_score=None,
          eval_metric='logloss', early_stopping_rounds=100, verbose=100,
          feature_name='auto', callbacks=None
         )
print(); print(model)

Training until validation scores don't improve for 100 rounds
[100]	Train's multi_logloss: 1.80145	Test's multi_logloss: 1.81396
[200]	Train's multi_logloss: 1.51399	Test's multi_logloss: 1.53268
[300]	Train's multi_logloss: 1.30804	Test's multi_logloss: 1.33125
[400]	Train's multi_logloss: 1.14936	Test's multi_logloss: 1.17629
[500]	Train's multi_logloss: 1.02218	Test's multi_logloss: 1.05232
[600]	Train's multi_logloss: 0.917663	Test's multi_logloss: 0.950798
[700]	Train's multi_logloss: 0.830611	Test's multi_logloss: 0.866422
[800]	Train's multi_logloss: 0.756838	Test's multi_logloss: 0.795147
[900]	Train's multi_logloss: 0.69424	Test's multi_logloss: 0.734903
[1000]	Train's multi_logloss: 0.640888	Test's multi_logloss: 0.683713
[1100]	Train's multi_logloss: 0.595098	Test's multi_logloss: 0.640035
[1200]	Train's multi_logloss: 0.555562	Test's multi_logloss: 0.602569
[1300]	Train's multi_logloss: 0.521335	Test's multi_logloss: 0.570394
[1400]	Train's multi_logloss: 0.491489	Test's mu

In [20]:
# make predictions
train_y_pred = model.predict_proba(X_test)
real_y_pred = model.predict_proba(test_df)

In [21]:
#log_loss
log_loss(y_test,train_y_pred,labels= list(set(y_test)))

0.33849746226503635

In [22]:
feature_df = pd.DataFrame(model.feature_importances_, index= train_X.columns,columns= ['importance']).sort_values('importance',ascending = False)
feature_df.head(10)

,importance
fiberID,757697
modelMag_r-i,581363
modelMag_u-g,562045
psfMag_u-g,559940
fiberMag_r-i,525851
modelMag_g-r,515664
psfMag_g-r,510421
fiberMag_u-g,509055
modelMag_i-z,505958
fiberMag_i-z,498815


In [23]:
submission = pd.DataFrame(data=real_y_pred, columns=sample_submission_df.columns, index=sample_submission_df.index)
loss_v=log_loss(y_test,train_y_pred,labels= list(set(y_test)))
submission.to_csv(f'submission_lightbgm_cv_10000iter_({loss_v}).csv', index=True)

In [331]:
# #최적의 변수를 찾기 위한 KBest 함수 불러오기
# from sklearn.feature_selection import SelectKBest
# model_kbest = SelectKBest()
from sklearn.pipeline import Pipeline
lgbm_c = lgb.LGBMClassifier()
#파이프라인 구축
pipe_lgbm = Pipeline([('lgbm',lgbm_c)])
#파라미터 생성
params_lgbm = {
    "lgbm__max_depth": [9,10],
    "lgbm__learning_rate": [0.001, 0.003],
    "lgbm__num_leaves": [300,500],
    "lgbm__n_estimators": [500,1500],
    "lgbm__random_state": [1059],
    "lgbm__subsample_for_bin": [50000,200000],
    "lgbm__objective": ['multiclass'],
    "lgbm__num_class": [train_y.nunique()],
    "lgbm__metric": [['multi_logloss','multi_logloss']],
    "lgbm__reg_alpha": [0.0, 1.0],
    "lgbm__reg_lambda": [0.0, 1.0],
    "lgbm__feature_fraction": [0.85], 
    "lgbm__bagging_fraction": [0.85], 
    "lgbm__bagging_freq": [20],
}

In [332]:
#최적의 파라미터 탐색
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipe_lgbm, param_grid = params_lgbm,
                          cv=2, n_jobs=-1, verbose=10)

In [333]:
grid_search.fit(X_train, y_train)
lgbm_grid = grid_search.best_estimator_
lgbm_grid

Fitting 2 folds for each of 128 candidates, totalling 256 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 27.0min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 45.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 53.6min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 103.3min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 144.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 219.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 262.3min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 290.7min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 324.0min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 411.9min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 507.6min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 566.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 601.2min
[Parallel(n_jobs=-1)]: Done 173 tasks      | e

Pipeline(memory=None,
         steps=[('lgbm',
                 LGBMClassifier(bagging_fraction=0.85, bagging_freq=20,
                                boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, feature_fraction=0.85,
                                importance_type='split', learning_rate=0.003,
                                max_depth=10,
                                metric=['multi_logloss', 'multi_logloss'],
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=1500,
                                n_jobs=-1, num_class=19, num_leaves=300,
                                objective='multiclass', random_state=1059,
                                reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                subsample=1.0, subsample_for_bin=200000,
                                subsample_freq=0))],
         verbose=False)

In [334]:
print(grid_search.best_score_)

0.8783559448165321


In [341]:
grid_search.best_params_

{'lgbm__bagging_fraction': 0.85,
 'lgbm__bagging_freq': 20,
 'lgbm__feature_fraction': 0.85,
 'lgbm__learning_rate': 0.001,
 'lgbm__max_depth': 9,
 'lgbm__metric': ['multi_logloss', 'multi_logloss'],
 'lgbm__n_estimators': 500,
 'lgbm__num_class': 19,
 'lgbm__num_leaves': 300,
 'lgbm__objective': 'multiclass',
 'lgbm__random_state': 1059,
 'lgbm__reg_alpha': 0.0,
 'lgbm__reg_lambda': 0.0,
 'lgbm__subsample_for_bin': 50000}